In [3]:
# !pip install ipywidgets

In [4]:
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# 🎙️ Pyannote Speaker Diarization 코드 설명

이 코드는 Hugging Face의 **pyannote.audio** 라이브러리를 사용하여  
오디오 파일(`audio.wav`)에서 **화자 분리(Speaker Diarization)**를 수행하는 예제입니다.  

---

## 주요 흐름

1. **환경 변수 불러오기**
   - `.env` 파일에 저장된 `HUGGINGFACE_ACCESS_TOKEN`을 불러와서 Hugging Face 모델 접근에 사용합니다.

2. **Pipeline 로드**
   - `pyannote/speaker-diarization-3.1` 모델을 불러옵니다.
   - 최신 버전의 `pyannote.audio`(3.x 계열)에서는 `plda` 같은 파라미터는 필요하지 않습니다.
   - GPU(CUDA)가 가능하다면 자동으로 GPU에서 실행됩니다.

3. **오디오 파일 경로 설정**
   - `os.getcwd()`를 사용해 현재 작업 디렉토리에서 `audio.wav` 파일을 불러옵니다.

4. **Diarization 실행**
   - `pipeline(audio_file, hook=hook)`을 실행하면 화자 분리 결과(`Annotation` 객체)가 반환됩니다.
   - `ProgressHook`을 사용하면 진행 상황이 표시됩니다.

5. **결과 출력**
   - 결과는 `output.itertracks(yield_label=True)`로 순회할 수 있습니다.
   - 여기서 각 변수의 의미는 다음과 같습니다:

---

## 출력 구조 설명

```python
for turn, _, speaker in output.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")
```

-turn

   -Segment 객체 (시작/끝 시간 정보를 담음)

   -예: Segment(0.2, 1.5)

   -접근: turn.start, turn.end

_ (track id)

   -동일 speaker 안에서 여러 트랙을 구분할 때 쓰이는 ID

   -특별히 사용하지 않으므로 _로 무시

-speaker

   -화자 레이블 (문자열)

   -예: "SPEAKER_00", "SPEAKER_01"

# 예시 출력

start=0.2s stop=1.5s speaker_SPEAKER_00

start=1.8s stop=3.9s speaker_SPEAKER_01

start=4.2s stop=5.7s speaker_SPEAKER_00


즉, 오디오의 각 구간별로 시작시간 / 종료시간 / 화자ID가 표시됩니다.

In [5]:
import os
# 현재 실행 중인 코드 파일 기준 절대경로
#오디오 파일 이름
AUDIO_PATH = "data/CabinetMeeting_economy_5p_12m_low.wav"
BASE_DIR = os.getcwd()
# 
audio_file = os.path.join(BASE_DIR, AUDIO_PATH)

In [6]:
import torch
from pyannote.audio import Pipeline
from pyannote.audio.pipelines.utils.hook import ProgressHook
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
HF_TOKEN = os.getenv("HUGGINGFACE_ACCESS_TOKEN")
#print("HF_TOKEN:", HF_TOKEN)

# Community-1 open-source speaker diarization pipeline
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=HF_TOKEN
)

C:\Users\손용훈\AppData\Local\Programs\Python\Python311\Lib\inspect.py:992: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):


In [7]:
import torch
print(torch.cuda.is_available())       # True
print(torch.version.cuda)              # 11.8 또는 12.1
print(torch.cuda.get_device_name(0))   # GPU 이름


True
12.1
NVIDIA GeForce RTX 3050


In [6]:


# CUDA가 있으면 GPU, 없으면 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline.to(device)

# 실행
with ProgressHook() as hook:
    output = pipeline(audio_file, hook=hook)

# 결과 출력
for turn, _, speaker in output.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")


Output()

e:\github\Module\sesacproject\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): 
degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by 
output numel). (Triggered internally at ..\aten\src\ATen\native\ReduceOps.cpp:1807.)
  std = sequences.std(dim=-1, correction=1)

start=0.0s stop=1.0s speaker_SPEAKER_04
start=1.0s stop=1.4s speaker_SPEAKER_00
start=1.7s stop=12.6s speaker_SPEAKER_00
start=11.5s stop=13.4s speaker_SPEAKER_04
start=13.4s stop=19.3s speaker_SPEAKER_00
start=19.3s stop=21.4s speaker_SPEAKER_04
start=22.1s stop=38.9s speaker_SPEAKER_00
start=39.3s stop=41.8s speaker_SPEAKER_04
start=42.8s stop=45.1s speaker_SPEAKER_04
start=46.0s stop=48.4s speaker_SPEAKER_04
start=49.3s stop=53.3s speaker_SPEAKER_04
start=54.7s stop=59.1s speaker_SPEAKER_04
start=61.2s stop=62.2s speaker_SPEAKER_04
start=62.4s stop=64.6s speaker_SPEAKER_04
start=64.9s stop=69.2s speaker_SPEAKER_00
start=69.1s stop=69.5s speaker_SPEAKER_04
start=69.5s stop=70.1s speaker_SPEAKER_00
start=70.1s stop=78.6s speaker_SPEAKER_04
start=75.3s stop=75.8s speaker_SPEAKER_00
start=76.5s stop=76.5s speaker_SPEAKER_00
start=76.6s stop=76.6s speaker_SPEAKER_00
start=79.3s stop=82.0s speaker_SPEAKER_04
start=82.5s stop=84.2s speaker_SPEAKER_00
start=85.0s stop=89.2s speaker_SPEAKER_

In [8]:


# CUDA가 있으면 GPU, 없으면 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline.to(device)

# 실행
with ProgressHook() as hook:
    output = pipeline(audio_file, hook=hook)

# 결과 출력
for turn, _, speaker in output.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")


Output()

e:\github\Module\sesacproject\Lib\site-packages\pyannote\audio\utils\reproducibility.py:74: ReproducibilityWarning:
TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(

e:\github\Module\sesacproject\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): 
degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by 
output numel). (Triggered internally at ..\aten\src\ATen\native\ReduceOps.cpp:1807.)
  std = sequences.std(dim=-1, correction=1)

start=0.0s stop=1.0s speaker_SPEAKER_04
start=1.0s stop=1.4s speaker_SPEAKER_00
start=1.7s stop=12.6s speaker_SPEAKER_00
start=11.5s stop=13.4s speaker_SPEAKER_04
start=13.4s stop=19.3s speaker_SPEAKER_00
start=19.3s stop=21.4s speaker_SPEAKER_04
start=22.1s stop=38.9s speaker_SPEAKER_00
start=39.3s stop=41.8s speaker_SPEAKER_04
start=42.8s stop=45.1s speaker_SPEAKER_04
start=46.0s stop=48.4s speaker_SPEAKER_04
start=49.3s stop=53.3s speaker_SPEAKER_04
start=54.7s stop=59.1s speaker_SPEAKER_04
start=61.2s stop=62.2s speaker_SPEAKER_04
start=62.4s stop=64.6s speaker_SPEAKER_04
start=64.9s stop=69.2s speaker_SPEAKER_00
start=69.1s stop=69.5s speaker_SPEAKER_04
start=69.5s stop=70.1s speaker_SPEAKER_00
start=70.1s stop=78.6s speaker_SPEAKER_04
start=75.3s stop=75.8s speaker_SPEAKER_00
start=76.5s stop=76.5s speaker_SPEAKER_00
start=76.6s stop=76.6s speaker_SPEAKER_00
start=79.3s stop=82.0s speaker_SPEAKER_04
start=82.5s stop=84.2s speaker_SPEAKER_00
start=85.0s stop=89.2s speaker_SPEAKER_

## API 사용

In [ ]:
import os
from pyannote.audio import Pipeline
from dotenv import load_dotenv

# ==============================
# 1. 환경 변수 로드
# ==============================
load_dotenv()
PYANNOTE_TOKEN = os.getenv("PYANNOTEAI_API_KEY")

if not PYANNOTE_TOKEN:
    raise ValueError("환경 변수에 PYANNOTEAI_API_KEY가 없습니다. .env 파일을 확인하세요.")

# ==============================
# 2. Precision-2 Speaker Diarization (서버 실행)
# ==============================
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-precision-2",
    token=PYANNOTE_TOKEN
)

# ==============================
# 3. 오디오 파일 경로
# ==============================
audio_file = os.path.join("data", "tiro_Test1_4People.wav")

# ==============================
# 4. 실행 (서버에서 처리)
# ==============================
output = pipeline(audio_file)  # pyannoteAI 서버에서 실행됨

# ==============================
# 5. 결과 출력
# ==============================
for turn, speaker in output.speaker_diarization:
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s {speaker}")
